### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, POPANE ####

In [13]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_popane as dr_p
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [22]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", 
        "Speech_Relax", "Speech_Anticipate"
    ],
    # [
    #     "Baseline_Rest", 
    #     "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
    #     "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    # ],
    ["BugBox_Break", "Speech_Break"],
    ["BugBox_Exposure", "Speech_Exposure"]
]

studies_popane = [
    "Study1",
    "Study3",
    # "Study4",
    "Study5"
]

model_phases_popane = [
    dr_p.Study1.ALL,
    dr_p.Study3.ALL,
    # dr_p.Study4.ALL,
    dr_p.Study5.ALL
]

apd_label_type = "Anxiety"
popane_label_type = "affect"

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

test_size = 0.8

`use_label_encoder` is deprecated in 1.7.0.


In [27]:
# TRAIN ON APD AND TEST ON POPANE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_p)
importlib.reload(dt)


for i, phases_apd in enumerate(model_phases_apd):
    for j, study in enumerate(studies_popane):
        print(f"POPANE PHASES {j} " + "-"*50)
        print(f"APD PHASES {i} " + "-"*50)
        x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        x_b, y_b = train.Train_POPANE.get_popane_data(study, metrics, model_phases_popane[j], verbose=False, label_type=popane_label_type)
        inds = pd.isnull(x_b).any(1).to_numpy().nonzero()[0]
        x_b = x_b.drop(inds, axis=0)
        y_b = y_b.drop(inds, axis=0)

        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # drop subjects with noisy data
        x_a = x_a[x_a['subject'] != 84.0]
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because POPANE doesn't have this feature
        y_a = y_a[y_a['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        print(y_a.loc[:, "label"].value_counts() + y_b.loc[:, "label"].value_counts())

        acc_results = {
            "SVM": [], 
            # "KNN": [],
            # "DT": [],
            "LogReg": [],
            # "Bayes": [],
            "XGB": []
        }
        reports = {
            "SVM": [], 
            # "KNN": [],
            # "DT": [],
            "LogReg": [],
            # "Bayes": [],
            "XGB": [],
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                print(f"Model evaluation metrics for {model_name}:")
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

APD PHASES 0 --------------------------------------------------
POPANE PHASES 0 --------------------------------------------------
0    429
1    397
Name: label, dtype: int64
SVM accuracy over 10 rounds: 0.4383324881676809
LogReg accuracy over 10 rounds: 0.7565792251653873
Model evaluation metrics for LogReg:
Precision: 0.0
Recall: 0.0
F1-score: 0.0
AUC score: 0.49372418164561493
XGB accuracy over 10 rounds: 0.6689814842434607
Model evaluation metrics for XGB:
Precision: 0.29160611398751046
Recall: 0.2875354800687203
F1-score: 0.2799740052009022
AUC score: 0.5373909032325053


APD PHASES 0 --------------------------------------------------
POPANE PHASES 1 --------------------------------------------------
1    584
0    292
Name: label, dtype: int64
SVM accuracy over 10 rounds: 0.5312692967934981
LogReg accuracy over 10 rounds: 0.6514786066711246
Model evaluation metrics for LogReg:
Precision: 0.6579032784347583
Recall: 0.9831842480524526
F1-score: 0.7881364069891644
AUC score: 0.496104

In [26]:
# TRAIN ON POPANE AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_p)
importlib.reload(dt)


for i, study in enumerate(studies_popane):
    for j, phases_apd in enumerate(model_phases_apd):
        print(f"POPANE PHASES {j} " + "-"*50)
        print(f"APD PHASES {i} " + "-"*50)
        x_a, y_a = train.Train_POPANE.get_popane_data(study, metrics, model_phases_popane[i], verbose=False, label_type=popane_label_type)
        x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        inds = pd.isnull(x_a).any(1).to_numpy().nonzero()[0]
        x_a = x_a.drop(inds, axis=0)
        y_a = y_a.drop(inds, axis=0)

        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # drop subjects with noisy data
        x_b = x_b[x_b['subject'] != 84.0]
        x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because POPANE doesn't have this feature
        y_b = y_b[y_b['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        print(y_a.loc[:, "label"].value_counts() + y_b.loc[:, "label"].value_counts())
        
        acc_results = {
            "SVM": [], 
            # "KNN": [],
            # "DT": [],
            "LogReg": [],
            # "Bayes": [],
            "XGB": []
        }
        reports = {
            "SVM": [], 
            # "KNN": [],
            # "DT": [],
            "LogReg": [],
            # "Bayes": [],
            "XGB": [],
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                print(f"Model evaluation metrics for {model_name}:")
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

POPANE PHASES 0 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
0    429
1    397
Name: label, dtype: int64
SVM accuracy over 10 rounds: 0.32649098144165273
LogReg accuracy over 10 rounds: 0.5952970993325595
XGB accuracy over 10 rounds: 0.602473165796415


POPANE PHASES 1 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
0    399
1    325
Name: label, dtype: int64
SVM accuracy over 10 rounds: 0.35041236984337126
LogReg accuracy over 10 rounds: 0.635268643343357
XGB accuracy over 10 rounds: 0.598327662058162


POPANE PHASES 2 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
0    355
1    267
Name: label, dtype: int64
SVM accuracy over 10 rounds: 0.3088296582414561
LogReg accuracy over 10 rounds: 0.6721477536938423
Model evaluation metrics for LogReg:
Precision: 0.7030274559138832
Recall: 0.91